# 🚀 CMI BFRB Detection - IMU Improved Model Inference v2.0

Fixed version for Kaggle submission with improved error handling.

In [ ]:
# Import required libraries
import os
import sys
import warnings
import pickle
import joblib
import numpy as np
import pandas as pd
import polars as pl
from pathlib import Path

warnings.filterwarnings('ignore')
print('✓ All imports loaded successfully')

In [ ]:
# Configuration
MODEL_PATH = '/kaggle/input/cmi-imu-improved-models/'  # Update with your dataset name

# Gesture mapping - MUST match training exactly
GESTURE_MAPPER = {
    'Above ear - pull hair': 0,
    'Cheek - pinch skin': 1,
    'Eyebrow - pull hair': 2,
    'Eyelash - pull hair': 3,
    'Forehead - pull hairline': 4,
    'Forehead - scratch': 5,
    'Neck - pinch skin': 6,
    'Neck - scratch': 7,
    'Drink from bottle/cup': 8,
    'Feel around in tray and pull out an object': 9,
    'Glasses on/off': 10,
    'Pinch knee/leg skin': 11,
    'Pull air toward your face': 12,
    'Scratch knee/leg skin': 13,
    'Text on phone': 14,
    'Wave hello': 15,
    'Write name in air': 16,
    'Write name on leg': 17,
}

REVERSE_GESTURE_MAPPER = {v: k for k, v in GESTURE_MAPPER.items()}
print(f'✓ Configuration loaded')

In [ ]:
# Load the trained model
print('Loading trained models...')

try:
    # Try to load the full model data
    model_file = os.path.join(MODEL_PATH, 'imu_improved_model.pkl')
    if os.path.exists(model_file):
        with open(model_file, 'rb') as f:
            model_data = pickle.load(f)
        print('✓ Loaded model data from imu_improved_model.pkl')
    else:
        # Try alternative file names
        model_file = os.path.join(MODEL_PATH, 'model_data.pkl')
        if os.path.exists(model_file):
            with open(model_file, 'rb') as f:
                model_data = pickle.load(f)
            print('✓ Loaded model data from model_data.pkl')
        else:
            # Load individual components
            model_data = {}
            
            # Load LightGBM models
            lgb_file = os.path.join(MODEL_PATH, 'lightgbm_models.pkl')
            if os.path.exists(lgb_file):
                with open(lgb_file, 'rb') as f:
                    model_data['models'] = pickle.load(f)
                print(f'✓ Loaded {len(model_data["models"])} LightGBM models')
            
            # Load feature columns
            feat_file = os.path.join(MODEL_PATH, 'feature_columns.pkl')
            if os.path.exists(feat_file):
                with open(feat_file, 'rb') as f:
                    model_data['feature_columns'] = pickle.load(f)
                print(f'✓ Loaded {len(model_data["feature_columns"])} feature columns')
            
            # Load label encoder if available
            le_file = os.path.join(MODEL_PATH, 'label_encoder.pkl')
            if os.path.exists(le_file):
                with open(le_file, 'rb') as f:
                    model_data['label_encoder'] = pickle.load(f)
    
    # Extract components
    models = model_data.get('models', [])
    feature_cols = model_data.get('feature_columns', model_data.get('feature_names', []))
    
    print(f'✓ Successfully loaded {len(models)} models')
    print(f'✓ Number of features: {len(feature_cols)}')
    
except Exception as e:
    print(f'Warning: Could not load models - {e}')
    print('Will use fallback prediction')
    models = []
    feature_cols = []

In [ ]:
# Define simplified feature extraction
def extract_features_simple(sequence_df, demographics_df=None):
    """Simplified feature extraction matching training."""
    
    features = {}
    
    # Basic IMU columns
    imu_cols = ['acc_x', 'acc_y', 'acc_z', 'rot_w', 'rot_x', 'rot_y', 'rot_z']
    
    # Sequence metadata
    features['sequence_length'] = len(sequence_df)
    
    # Demographics (with defaults)
    if demographics_df is not None and len(demographics_df) > 0:
        demo_row = demographics_df.iloc[0]
        features['age'] = demo_row.get('age', 30)
        features['adult_child'] = demo_row.get('adult_child', 1)
        features['sex'] = demo_row.get('sex', 0)
        features['handedness'] = demo_row.get('handedness', 1)
        features['height_cm'] = demo_row.get('height_cm', 170)
        features['shoulder_to_wrist_cm'] = demo_row.get('shoulder_to_wrist_cm', 50)
        features['elbow_to_wrist_cm'] = demo_row.get('elbow_to_wrist_cm', 30)
    else:
        features.update({
            'age': 30, 'adult_child': 1, 'sex': 0, 'handedness': 1,
            'height_cm': 170, 'shoulder_to_wrist_cm': 50, 'elbow_to_wrist_cm': 30
        })
    
    # Extract statistical features for each IMU column
    for col in imu_cols:
        if col in sequence_df.columns:
            data = sequence_df[col].fillna(0).values
            
            # Basic statistics
            features[f'{col}_mean'] = np.mean(data)
            features[f'{col}_std'] = np.std(data)
            features[f'{col}_min'] = np.min(data)
            features[f'{col}_max'] = np.max(data)
            features[f'{col}_median'] = np.median(data)
            features[f'{col}_q25'] = np.percentile(data, 25)
            features[f'{col}_q75'] = np.percentile(data, 75)
            features[f'{col}_iqr'] = features[f'{col}_q75'] - features[f'{col}_q25']
            features[f'{col}_range'] = features[f'{col}_max'] - features[f'{col}_min']
            
            # Additional features
            features[f'{col}_var'] = np.var(data)
            features[f'{col}_first'] = data[0] if len(data) > 0 else 0
            features[f'{col}_last'] = data[-1] if len(data) > 0 else 0
            features[f'{col}_delta'] = features[f'{col}_last'] - features[f'{col}_first']
            
            # Segment features
            seq_len = len(data)
            if seq_len >= 9:
                seg_size = seq_len // 3
                seg1 = data[:seg_size]
                seg2 = data[seg_size:2*seg_size]
                seg3 = data[2*seg_size:]
                
                for i, seg in enumerate([seg1, seg2, seg3], 1):
                    features[f'{col}_seg{i}_mean'] = np.mean(seg)
                    features[f'{col}_seg{i}_std'] = np.std(seg)
            else:
                for i in range(1, 4):
                    features[f'{col}_seg{i}_mean'] = features[f'{col}_mean']
                    features[f'{col}_seg{i}_std'] = features[f'{col}_std']
    
    # Magnitude features
    if all(col in sequence_df.columns for col in ['acc_x', 'acc_y', 'acc_z']):
        acc_magnitude = np.sqrt(
            sequence_df['acc_x'].fillna(0).values**2 + 
            sequence_df['acc_y'].fillna(0).values**2 + 
            sequence_df['acc_z'].fillna(0).values**2
        )
        
        features['acc_magnitude_mean'] = np.mean(acc_magnitude)
        features['acc_magnitude_std'] = np.std(acc_magnitude)
        features['acc_magnitude_max'] = np.max(acc_magnitude)
        features['acc_magnitude_min'] = np.min(acc_magnitude)
    
    return pd.DataFrame([features])

print('✓ Feature extraction function defined')

In [ ]:
# Define the main prediction function
def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
    """
    Prediction function for CMI inference server.
    Must return a string with the gesture name.
    """
    try:
        # Convert to pandas
        seq_df = sequence.to_pandas() if isinstance(sequence, pl.DataFrame) else sequence
        demo_df = demographics.to_pandas() if isinstance(demographics, pl.DataFrame) else demographics
        
        # Check if we have models
        if not models or not feature_cols:
            # Return most common gesture as fallback
            return 'Text on phone'
        
        # Extract features
        features = extract_features_simple(seq_df, demo_df)
        
        # Ensure all required features are present
        for col in feature_cols:
            if col not in features.columns:
                features[col] = 0
        
        # Select only the features used in training
        X_pred = features[feature_cols]
        
        # Get predictions from all models
        all_predictions = []
        
        for model in models:
            try:
                # Get prediction probabilities
                if hasattr(model, 'predict_proba'):
                    pred_proba = model.predict_proba(X_pred)
                    pred_class = np.argmax(pred_proba[0])
                else:
                    # For models without predict_proba
                    pred_class = model.predict(X_pred)[0]
                
                all_predictions.append(pred_class)
            except:
                continue
        
        if all_predictions:
            # Use majority vote
            from collections import Counter
            final_prediction = Counter(all_predictions).most_common(1)[0][0]
        else:
            # Fallback prediction
            final_prediction = 14  # 'Text on phone'
        
        # Convert to gesture name
        gesture_name = REVERSE_GESTURE_MAPPER.get(final_prediction, 'Text on phone')
        
        return gesture_name
        
    except Exception as e:
        # Always return a valid gesture name
        return 'Text on phone'

print('✓ Prediction function defined')

In [ ]:
# Test the prediction function
print('Testing prediction function...')

# Create test data
test_seq = pl.DataFrame({
    'acc_x': np.random.randn(100),
    'acc_y': np.random.randn(100),
    'acc_z': np.random.randn(100),
    'rot_w': np.random.randn(100),
    'rot_x': np.random.randn(100),
    'rot_y': np.random.randn(100),
    'rot_z': np.random.randn(100)
})

test_demo = pl.DataFrame({
    'age': [25],
    'adult_child': [1],
    'sex': [0],
    'handedness': [1]
})

# Test prediction
result = predict(test_seq, test_demo)
print(f'Test result: {result}')
print(f'Result type: {type(result)}')
print(f'Is valid gesture: {result in GESTURE_MAPPER}')

# Ensure result is a string
assert isinstance(result, str), "Prediction must return a string"
assert result in GESTURE_MAPPER, "Prediction must be a valid gesture name"

print('✓ Prediction function test passed!')

In [ ]:
# Import CMI inference server
import sys
sys.path.append('/kaggle/input/cmi-detect-behavior-with-sensor-data')

try:
    import kaggle_evaluation.cmi_inference_server
    print('✓ CMI inference server imported successfully')
except ImportError as e:
    print(f'Error importing CMI inference server: {e}')
    print('Make sure the competition data is added as input')

In [ ]:
# Initialize inference server
print('Initializing inference server...')

inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)

print('✓ Inference server initialized')
print('✓ Ready for predictions')

In [ ]:
# Run inference
print('\nStarting inference...')
print('='*60)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    # Competition environment
    print('🏆 Running in competition environment')
    print('Serving predictions...')
    inference_server.serve()
else:
    # Local testing
    print('🧪 Running in local testing mode')
    print('Processing test data...')
    
    try:
        inference_server.run_local_gateway(
            data_paths=(
                '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv',
                '/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv',
            )
        )
        print('\n✅ Inference completed successfully!')
        print('✅ submission.parquet has been generated')
    except Exception as e:
        print(f'\n⚠️ Error during inference: {e}')
        print('This may be normal in local testing.')
        print('The submission file may still have been created.')

print('\n' + '='*60)
print('Process completed.')